### Model
| model |
| --- |
| gpt-3.5-turbo-0125 |
| gpt-4-0125-preview |

In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1
)

In [6]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(content="You are a geography expert. And you only reply in {language}"),
    AIMessage(content="Hi my name is {name}!"),
    HumanMessage(content="서울과 도쿄의 거리는 얼마인가요. 너의 이름은 뭐니?"),
]

chat.predict_messages(messages)

AIMessage(content='서울과 도쿄의 직선 거리는 약 1,200km 정도입니다. 제 이름은 루카스입니다.')

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1
)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
    ]
)

prompt = template.format_messages(
    language = "Korean",
    name = "AiGENDA",
    country_a = "Korea",
    country_b = "USA"
)

chat.predict_messages(prompt).content

'한국과 미국 사이의 거리는 대략 10,000km입니다. 제 이름은 AiGENDA입니다.'

### 3.3 OutputParser and LCEL 

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        
        return list(map(str.strip, items))

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0
)

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "{question}"),
    ]
)

prompt = template.format_messages(
    name = "aigenda",
    language = "Korean",
    max_items = 10,
    question = "2024 한국 축구 국가대표 맴버",
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

['1. 손흥민',
 '2. 황의조',
 '3. 김민재',
 '4. 김영권',
 '5. 김민규',
 '6. 김진수',
 '7. 김영광',
 '8. 이재성',
 '9. 황인범',
 '10. 김승대']

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items}. Do NOT reply with anything else. And you only reply in {language}."),
        ("ai", "Hi my name is {name}!"),
        ("human", "{question}"),
    ]
)

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0
)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        
        return list(map(str.strip, items))

chain = template | chat | CommaOutputParser()    

chain.invoke({
    "name":"aigenda",
    "language":"Korean",
    "max_items":5,
    "question":"2024 한국 축구 국가대표 맴버",
})

['손흥민', '황의조', '김민재', '김영권', '김민재']

### 3.4 Chaining Chains

In [32]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a list generating machine. \
         Everything you are asked will be answered with a comma separated list of max 5 in lowercase. \
         Do NOT reply with anything else."),
        # ("human", "{question}"),
        ("human", "I want to cook {cuisine} food."),
    ]
)

veg_chef_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a vegetarian chef specialized on making traditional recipies vegetarian. \
         You find alternative ingredients and explain their preparation. \
         You don't radically modify the recipe. \
         If there is no alternative for a food just say you don't know how to replace it. \
         And you only reply in Korean"),
        ("human", "{recipe}")
    ]
)

chef_chain = chef_prompt | chat

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({"cuisine":"korea"})



kimchi, bulgogi, bibimbap, tteokbokki김치는 대체할 수 있는 재료가 없습니다. 

불고기를 채식주의자용으로 만들려면 대체 고기로 대신할 수 있습니다. 대체 고기로는 대부분 식물성 단백질이나 버섯을 사용합니다. 대체 고기를 사용할 때는 불고기 소스와 함께 볶아내면 됩니다.

비빔밥은 채소나 과일을 활용하여 대체할 수 있습니다. 대체 재료로는 아보카도, 콩, 토마토, 당근, 오이 등을 사용할 수 있습니다. 이를 채소와 함께 밥 위에 올려 비빔밥 소스와 함께 섞어먹으면 됩니다.

떡볶이는 떡 대신에 떡 대신에 고구마, 양배추, 어묵, 버섯 등을 사용하여 대체할 수 있습니다. 떡 대신에 이러한 채소나 버섯을 썰어서 떡볶이 소스와 함께 볶아내면 됩니다.

AIMessageChunk(content='김치는 대체할 수 있는 재료가 없습니다. \n\n불고기를 채식주의자용으로 만들려면 대체 고기로 대신할 수 있습니다. 대체 고기로는 대부분 식물성 단백질이나 버섯을 사용합니다. 대체 고기를 사용할 때는 불고기 소스와 함께 볶아내면 됩니다.\n\n비빔밥은 채소나 과일을 활용하여 대체할 수 있습니다. 대체 재료로는 아보카도, 콩, 토마토, 당근, 오이 등을 사용할 수 있습니다. 이를 채소와 함께 밥 위에 올려 비빔밥 소스와 함께 섞어먹으면 됩니다.\n\n떡볶이는 떡 대신에 떡 대신에 고구마, 양배추, 어묵, 버섯 등을 사용하여 대체할 수 있습니다. 떡 대신에 이러한 채소나 버섯을 썰어서 떡볶이 소스와 함께 볶아내면 됩니다.')

## Model IO

https://python.langchain.com/docs/modules/model_io/

https://python.langchain.com/docs/modules/data_connection/

### 4.1 FewShotPromptTemplate

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: 파리
        Language: 불어
        Food: 와인, 치즈
        Currency: 유로화
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: 로마
        Language: 이태리어
        Food: 피자, 파스타
        Currency: 유로화
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: 아테네
        Language: 그리스어
        Food: 수블라키 and 페타 치즈
        Currency: 유로화
        """
    },
]

example_prompt = PromptTemplate.from_template("Human : {question}\nAI : {answer}")
    
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Human : What do you know about {country}?",
    input_variables=["country"],
)

chain = prompt | chat

chain.invoke({
    "country" : "아이슬란드"
})

AI : 
        I know this:
        Capital: 레이캬비크
        Language: 아이슬란드어
        Food: 스모크드 피쉬, 람
        Currency: 아이슬란드 크로나

AIMessageChunk(content='AI : \n        I know this:\n        Capital: 레이캬비크\n        Language: 아이슬란드어\n        Food: 스모크드 피쉬, 람\n        Currency: 아이슬란드 크로나')

### 4.2 FewShotChatMessagePromptTemplate 

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "country": "France?",
        "answer": """
        Here is what I know:
        Capital: 파리
        Language: 불어
        Food: 와인, 치즈
        Currency: 유로화
        """,
    },
    {
        "country": "Italy?",
        "answer": """
        I know this:
        Capital: 로마
        Language: 이태리어
        Food: 피자, 파스타
        Currency: 유로화
        """
    },
    {
        "country": "Greece?",
        "answer": """
        I know this:
        Capital: 아테네
        Language: 그리스어
        Food: 수블라키 and 페타 치즈
        Currency: 유로화
        """
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

# 예시 문장과 실제 예시를 보여줌   
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# system : 전달 인자, example_prompt : 예시 문장과 실제 예시 프롬프트, human : 물어볼 메시지
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

print(f"3 : {final_prompt}")

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})

3 : input_variables=['country'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a geography expert, you give short answers.')), FewShotChatMessagePromptTemplate(examples=[{'country': 'France?', 'answer': '\n        Here is what I know:\n        Capital: 파리\n        Language: 불어\n        Food: 와인, 치즈\n        Currency: 유로화\n        '}, {'country': 'Italy?', 'answer': '\n        I know this:\n        Capital: 로마\n        Language: 이태리어\n        Food: 피자, 파스타\n        Currency: 유로화\n        '}, {'country': 'Greece?', 'answer': '\n        I know this:\n        Capital: 아테네\n        Language: 그리스어\n        Food: 수블라키 and 페타 치즈\n        Currency: 유로화\n        '}], example_prompt=ChatPromptTemplate(input_variables=['answer', 'country'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], template='What do you know about {country}?')), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=['answer'], 


        I know this:
        Capital: 방콕
        Language: 태국어
        Food: 태국 카레, 생선 찌개
        Currency: 태국 바트
        

AIMessageChunk(content='\n        I know this:\n        Capital: 방콕\n        Language: 태국어\n        Food: 태국 카레, 생선 찌개\n        Currency: 태국 바트\n        ')

### 4.3 LengthBasedExampleSelector

example의 수를 제한해서 비용을 줄일 수 있는 방법

In [35]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()],
)

class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: 파리
        Language: 불어
        Food: 와인, 치즈
        Currency: 유로화
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: 로마
        Language: 이태리어
        Food: 피자, 파스타
        Currency: 유로화
        """
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: 아테네
        Language: 그리스어
        Food: 수블라키 and 페타 치즈
        Currency: 유로화
        """
    },
]

example_prompt = PromptTemplate.from_template("Human : {question}\nAI : {answer}")

example_selector = RandomExampleSelector(
    examples = examples,
)
    
prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    example_selector = example_selector,
    suffix = "Human : What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="브라질")

'Human : What do you know about Italy?\nAI : \n        I know this:\n        Capital: 로마\n        Language: 이태리어\n        Food: 피자, 파스타\n        Currency: 유로화\n        \n\nHuman : What do you know about 브라질?'

### 4.4 Serialization and Composition

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

prompt = load_prompt("./prompt.json")

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature = 0.1,
    streaming=True,
    callbacks = [StreamingStdOutCallbackHandler()],
)

prompt.format(country="xxx")

'What is the capital of xxx'

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
    """
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
    """
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)

Arrrrg matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, it be a meal fit for a pirate like meself! Arrrrg!

AIMessageChunk(content="Arrrrg matey! Me favorite grub be a hearty plate o' salted beef and hardtack! Aye, it be a meal fit for a pirate like meself! Arrrrg!")

### 4.5 Caching

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))
set_debug(True)


chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pizza")

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make italian pizza"
  ]
}
[llm/end] [1:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make Italian pizza, you will need the following ingredients:\n\n- 2 1/4 cups of all-purpose flour\n- 1 teaspoon of salt\n- 1 teaspoon of sugar\n- 1 packet of active dry yeast\n- 1 cup of warm water\n- 2 tablespoons of olive oil\n- Tomato sauce\n- Mozzarella cheese\n- Toppings of your choice (such as pepperoni, mushrooms, bell peppers, etc.)\n\nHere is a step-by-step guide to making Italian pizza:\n\n1. In a large mixing bowl, combine the flour, salt, and sugar. In a separate small bowl, dissolve the yeast in the warm water and let it sit for about 5 minutes until it becomes frothy.\n\n2. Pour the yeast mixture and olive oil into the flour mixture and stir until a dough forms. Knead the dough on a floured surface for about 5-7 minutes until it becom

'To make Italian pizza, you will need the following ingredients:\n\n- 2 1/4 cups of all-purpose flour\n- 1 teaspoon of salt\n- 1 teaspoon of sugar\n- 1 packet of active dry yeast\n- 1 cup of warm water\n- 2 tablespoons of olive oil\n- Tomato sauce\n- Mozzarella cheese\n- Toppings of your choice (such as pepperoni, mushrooms, bell peppers, etc.)\n\nHere is a step-by-step guide to making Italian pizza:\n\n1. In a large mixing bowl, combine the flour, salt, and sugar. In a separate small bowl, dissolve the yeast in the warm water and let it sit for about 5 minutes until it becomes frothy.\n\n2. Pour the yeast mixture and olive oil into the flour mixture and stir until a dough forms. Knead the dough on a floured surface for about 5-7 minutes until it becomes smooth and elastic.\n\n3. Place the dough in a greased bowl, cover it with a clean kitchen towel, and let it rise in a warm place for about 1-2 hours, or until it has doubled in size.\n\n4. Preheat your oven to 475°F (245°C) and place 

### 4.6 Serialization

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

# from langchain.globals import set_llm_cache, set_debug
# from langchain.cache import SQLiteCache

# set_llm_cache(SQLiteCache("cache.db"))
# set_debug(True)

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125",
    temperature=0.1,
    max_tokens= 1000,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for toast")
    b = chat.predict("What is the recipe for egg sandwich")
    print(a, "\n")
    print(b, "\n")
    print(usage)

To make toast, you will need the following ingredients and equipment:

Ingredients:
- Bread slices
- Butter or margarine (optional)

Equipment:
- Toaster or toaster oven

Instructions:
1. Plug in your toaster or toaster oven and set it to the desired level of toasting (light, medium, or dark).
2. Place the bread slices into the toaster slots or on the toaster oven tray.
3. Press down the lever on the toaster or start the toaster oven.
4. Wait 

Ingredients:
- 2 slices of bread
- 2 eggs
- 1 tablespoon of butter
- Salt and pepper to taste
- Optional toppings: cheese, bacon, avocado, tomato, spinach

Instructions:
1. Heat a non-stick skillet over medium heat and melt the butter.
2. Crack the eggs into the skillet and season with salt and pepper.
3. Cook the eggs to your desired doneness, either scrambled, fried, or poached.
4. While the eggs 

Tokens Used: 227
	Prompt Tokens: 27
	Completion Tokens: 200
Successful Requests: 2
Total Cost (USD): $0.0
